In [1]:
import igimf as ii

In [2]:
import igimf.instance as inst

In [3]:
import numpy as np
mstar_l=0.07
mstar_u=150
mstar_v = np.logspace(np.log10(mstar_l),np.log10(mstar_u), num=50)


### This is what happens when you run instance

In [4]:
massfrac, SFR, Mtot, stellar_IMF, ecl_MF, t_IGIMF = inst.main()


At a time t characterized by:
SFR=1.00e+00
and 
Mtot=1.00e+10


dictionary of an e.cl.'s stellar IMF

{
    "solar_metallicity": 0.0134,
    "delta_alpha": 63,
    "m_star_max": 150,
    "m_star_min": 0.07,
    "M_ecl_max": 10000000000.0,
    "M_ecl_min": 5,
    "delta_t": 10000000.0,
    "SFR": 1.0,
    "Mtot": 10000000.0,
    "metal_mass_fraction": 0.00134,
    "metallicity": -1.0,
    "M_ecl": 10000000000.0,
    "alpha_1": 0.5402199999999999,
    "alpha_2": 1.54022,
    "alpha_3": 0.6851950000000002,
    "k_star": 18092236.188298263,
    "m_max": 149.9999982877995,
    "IMF_func": "<numpy.vectorize object at 0x113dd28b0>",
    "IMF_weighted_func": "<numpy.vectorize object at 0x113c1a0d0>"
}


dictionary of a galaxy's ECMF

{
    "solar_metallicity": 0.0134,
    "delta_alpha": 63,
    "m_star_max": 150,
    "m_star_min": 0.07,
    "M_ecl_max": 10000000000.0,
    "M_ecl_min": 5,
    "delta_t": 10000000.0,
    "SFR": 1.0,
    "Mtot": 10000000.0,
    "metal_mass_fraction": 0.00134,
   

In [6]:
t_IGIMF.IGIMF_func(mstar_v)

array([1.38060733e+07, 1.26866312e+07, 1.16579571e+07, 1.07126914e+07,
       9.84407090e+06, 9.04588105e+06, 8.31241108e+06, 7.63841327e+06,
       7.01906544e+06, 6.44993639e+06, 5.92695420e+06, 5.44637713e+06,
       5.00476685e+06, 4.29337734e+06, 3.37361676e+06, 2.65089443e+06,
       2.08299927e+06, 1.63494796e+06, 1.16199139e+06, 8.26434914e+05,
       5.88208480e+05, 4.18968070e+05, 2.98653582e+05, 2.13059930e+05,
       1.52122375e+05, 1.08705514e+05, 7.77475907e+04, 5.56555806e+04,
       3.98773395e+04, 2.85988311e+04, 2.05297500e+04, 1.47516478e+04,
       1.06102764e+04, 7.63922759e+03, 5.50573828e+03, 3.97220126e+03,
       2.86881200e+03, 2.07411603e+03, 1.50116488e+03, 1.08765637e+03,
       6.23904045e+02, 4.57722471e+02, 3.36090189e+02, 2.46987603e+02,
       1.81658791e+02, 1.33719743e+02, 9.06872393e+01, 6.71733439e+01,
       4.66932866e+01, 0.00000000e+00])

In [10]:
50*50*42/60/60

29.166666666666668